In [ ]:
from google.colab import drive
import os
import pandas as pd
import json

In [ ]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/shinhan_dataset/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.listdir(path)

['big_data_set3_f.csv',
 'big_data_set1_f.csv',
 'big_data_set2_f.csv',
 '신한',
 'Copy of 회복탄력성_지수_계산.ipynb',
 'api_key.txt',
 '.env',
 '신한usingMilvus',
 '신한v2',
 'Untitled1.ipynb']

In [ ]:
df1=pd.read_csv(path+"big_data_set1_f.csv", encoding='cp949')
df2=pd.read_csv(path+"big_data_set2_f.csv", encoding='cp949')
df3=pd.read_csv(path+"big_data_set3_f.csv", encoding='cp949')

In [ ]:
column_map_df1 = {
    'ENCODED_MCT': '가맹점구분번호',
    'MCT_BSE_AR': '가맹점주소',
    'MCT_NM': '가맹점명',
    'MCT_BRD_NUM': '브랜드구분코드',
    'MCT_SIGUNGU_NM': '가맹점지역',
    'HPSN_MCT_ZCD_NM': '업종',
    'HPSN_MCT_BZN_CD_NM': '상권',
    'ARE_D': '개설일',
    'MCT_ME_D': '폐업일'}
df1 = df1.rename(columns=column_map_df1)

In [ ]:
df_joined = pd.merge(df2, df3, on=['ENCODED_MCT', 'TA_YM'], how='inner')
df_joined['MCT_OPE_MS_CN'] = df_joined['MCT_OPE_MS_CN'].str[2:]
df_joined['RC_M1_SAA'] = df_joined['RC_M1_SAA'].str[2:]
df_joined['RC_M1_TO_UE_CT'] = df_joined['RC_M1_TO_UE_CT'].str[2:]
df_joined['RC_M1_UE_CUS_CN'] = df_joined['RC_M1_UE_CUS_CN'].str[2:]
df_joined['RC_M1_AV_NP_AT'] = df_joined['RC_M1_AV_NP_AT'].str[2:]
df_joined['APV_CE_RAT'] = df_joined['APV_CE_RAT'].str[2:]

In [ ]:
column_map_join = {
    'ENCODED_MCT': '가맹점구분번호',
    'TA_YM': '기준년월',
    'MCT_OPE_MS_CN': '가맹점 운영개월수 구간',
    'RC_M1_SAA': '매출금액 구간',
    'RC_M1_TO_UE_CT': '매출건수 구간',
    'RC_M1_UE_CUS_CN': '유니크 고객 수 구간',
    'RC_M1_AV_NP_AT': '객단가 구간',
    'APV_CE_RAT': '취소율 구간',
    'DLV_SAA_RAT': '배달매출금액 비율',
    'M1_SME_RY_SAA_RAT': '동일 업종 매출금액 비율',
    'M1_SME_RY_CNT_RAT': '동일 업종 매출건수 비율',
    'M12_SME_RY_SAA_PCE_RT': '동일 업종 내 매출수위비율',
    'M12_SME_BZN_SAA_PCE_RT': '동일 상권 내 매출수위비율',
    'M12_SME_RY_ME_MCT_RAT': '동일 업종 내 해지 가맹점 비중',
    'M12_SME_BZN_ME_MCT_RAT': '동일 상권 내 해지 가맹점 비중',
    'M12_MAL_1020_RAT': '남성 20대이하',
    'M12_MAL_30_RAT': '남성 30대',
    'M12_MAL_40_RAT': '남성 40대',
    'M12_MAL_50_RAT': '남성 50대',
    'M12_MAL_60_RAT': '남성 60대이상',
    'M12_FME_1020_RAT': '여성 20대이하',
    'M12_FME_30_RAT': '여성 30대',
    'M12_FME_40_RAT': '여성 40대',
    'M12_FME_50_RAT': '여성 50대',
    'M12_FME_60_RAT': '여성 60대이상',
    'MCT_UE_CLN_REU_RAT': '재방문 고객 비중',
    'MCT_UE_CLN_NEW_RAT': '신규 고객 비중',
    'RC_M1_SHC_RSD_UE_CLN_RAT': '거주 인구 고객 비율',
    'RC_M1_SHC_WP_UE_CLN_RAT': '직장 이용 고객 비율',
    'RC_M1_SHC_FLP_UE_CLN_RAT': '유동인구 이용 고객 비율'
}


df_joined = df_joined.rename(columns=column_map_join)

In [ ]:
store_cd = df1['가맹점구분번호']
store_cd_list_total = store_cd.tolist()
print(len(store_cd_list_total))

4185


In [ ]:
k = 2000
store_cd_list1 = store_cd_list_total[:k]
store_cd_list2 = store_cd_list_total[k:]

In [ ]:
st_final_lst1=[]
st_final_lst2=[]

for cd in store_cd_list1:
  df_info = df1[df1['가맹점구분번호']==cd]
  st_info_dict_list = df_info.to_dict(orient='list')
  st_info_dict = {
    key: value[0]
    for key, value in st_info_dict_list.items()
  }


  df_usage = df_joined[df_joined['가맹점구분번호']==cd].sort_values(by='기준년월')
  df_usage_filtered = df_usage[['가맹점 운영개월수 구간','매출금액 구간','매출건수 구간','유니크 고객 수 구간','객단가 구간','취소율 구간','배달매출금액 비율','동일 업종 매출금액 비율', \
                          '동일 업종 매출건수 비율','동일 업종 내 매출수위비율','동일 상권 내 매출수위비율','동일 업종 내 해지 가맹점 비중','동일 상권 내 해지 가맹점 비중',\
                          '남성 20대이하','남성 30대','남성 40대','남성 50대','남성 60대이상','여성 20대이하','여성 30대','여성 40대','여성 50대','여성 60대이상','재방문 고객 비중',\
                          '신규 고객 비중','거주 인구 고객 비율','직장 이용 고객 비율','유동인구 이용 고객 비율']]
  st_usage_dict = df_usage_filtered.to_dict(orient='list')

  json_final_dict = st_info_dict | st_usage_dict
  json_final = json.dumps(json_final_dict, ensure_ascii=False, indent=4)
  st_final_lst1.append(json_final)



for cd in store_cd_list2:
  df_info = df1[df1['가맹점구분번호']==cd]
  st_info_dict_list = df_info.to_dict(orient='list')
  st_info_dict = {
    key: value[0]
    for key, value in st_info_dict_list.items()
  }


  df_usage = df_joined[df_joined['가맹점구분번호']==cd].sort_values(by='기준년월')
  df_usage_filtered = df_usage[['가맹점 운영개월수 구간','매출금액 구간','매출건수 구간','유니크 고객 수 구간','객단가 구간','취소율 구간','배달매출금액 비율','동일 업종 매출금액 비율', \
                          '동일 업종 매출건수 비율','동일 업종 내 매출수위비율','동일 상권 내 매출수위비율','동일 업종 내 해지 가맹점 비중','동일 상권 내 해지 가맹점 비중',\
                          '남성 20대이하','남성 30대','남성 40대','남성 50대','남성 60대이상','여성 20대이하','여성 30대','여성 40대','여성 50대','여성 60대이상','재방문 고객 비중',\
                          '신규 고객 비중','거주 인구 고객 비율','직장 이용 고객 비율','유동인구 이용 고객 비율']]
  st_usage_dict = df_usage_filtered.to_dict(orient='list')

  json_final_dict = st_info_dict | st_usage_dict
  json_final = json.dumps(json_final_dict, ensure_ascii=False, indent=4)
  st_final_lst2.append(json_final)


In [ ]:
print(len(st_final_lst1))
print(len(st_final_lst2))

2000
2185


In [ ]:
store_nm = df1['가맹점명']

store_nm_lst_tmp1 = store_nm[:k].tolist()
store_nm_lst1 = [s[:4] if len(s) > 4 else s for s in store_nm_lst_tmp1]
print(store_nm_lst1)
print(len(store_nm_lst1))

store_nm_lst_tmp2 = store_nm[k:].tolist()
store_nm_lst2 = [s[:4] if len(s) > 4 else s for s in store_nm_lst_tmp2]
print(store_nm_lst2)
print(len(store_nm_lst2))

['성우**', '대보**', '대용**', '통일**', '한울**', '대광**', '미림**', '호남*', '고향**', '북*', '좋은**', '별난**', '행당**', '이혜**', '정*', '조은**', '왕십**', '광한**', '멕시**', '도미**', '현대**', '청계**', '한양**', '한일**', '세화**', '대성**', '광*', '희망**', '한일**', '세계**', '파리**', '진*', '삼대**', '대구*', '원조**', '훈희**', '몽실**', '순대**', '동원**', '마장**', '와이**', '맛드*', '피자**', '담바*', '델리**', '본토**', '동부**', '고향**', '풀무**', '단아**', '영*', '에스**', '종로**', '만두**', '코페*', '아폴**', '무탁**', '정은**', '맛나**', '설빙**', '70**', '오가**', '극동**', '전주**', '양지**', '장군**', '돈까*', '돔카*', '한양**', '한양**', '김밥**', '대림**', '닭한**', '귀일**', '해장**', '교촌**', '광주**', '와규**', '참나**', '써브**', '대*', '돼지*', '장어**', '놀란*', '토리**', '성일**', '맘스**', '한솥**', '동방**', '크로**', '스마*', '철*', '인덕**', '파파**', '왕십**', '벽산**', '춘천**', '가든**', '둘둘**', '이흥**', '왕짜*', '맛닭**', '삼오**', '찐짜**', '20**', '빙아**', '뚱이**', '롯데**', '두레**', '먹거**', '온달**', '백미*', '진미**', '은하*', '덕흥**', '비타**', '강정**', '미스**', '예*', '포마**', '사와**', '마테**', '해*', '유어**', '금호**', '필라**', '만족**', '대중**', '장미**'

In [ ]:
!pip install langchain langchain-community pymilvus langchain_huggingface

In [ ]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from pymilvus import MilvusClient, CollectionSchema, FieldSchema, DataType
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
import time
from dotenv import load_dotenv

# .env 파일에서 환경 변수(api key, URL 등) 로드
load_dotenv(dotenv_path=path+'.env')

True

In [ ]:

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
ZILLIZ_URI = os.getenv("ZILLIZ_URI")
ZILLIZ_TOKEN = os.getenv("ZILLIZ_TOKEN")
HF_TOKEN = os.environ.get("HF_TOKEN")

EMBEDDING_MODEL = "jhgan/ko-sroberta-nli"
EMBEDDING_DIMENSION =768
COLLECTION_NAME='shinahn_collection_hf'

임베딩 모델 테스트

In [ ]:
# LangChain용 Hugging Face Endpoint Embedding 생성
embedding_model = HuggingFaceEndpointEmbeddings(
    model=EMBEDDING_MODEL,
    # task="feature-extraction",
    huggingfacehub_api_token=HF_TOKEN,
)

# 텍스트 리스트
texts = [
    "LangChain seamlessly integrates with Hugging Face API.",
    "Embeddings help capture semantic meaning of text."
]

# 임베딩 생성
vectors = embedding_model.embed_documents(texts)


벡터디비 생성 준비

In [ ]:
try:
    # Zilliz Cloud (MilvusClient) 연결
    milvus_client = MilvusClient(uri=ZILLIZ_URI, token=ZILLIZ_TOKEN)
    print("✅ Zilliz Cloud에 연결되었습니다.")

    # LangChain OpenAIEmbeddings 초기화 (🚨 변경된 부분)
    # LangChain은 자동으로 OPENAI_API_KEY 환경 변수를 확인합니다.
    # 여기서는 명시적으로 key와 model을 전달합니다.
    # embedding_model = OpenAIEmbeddings(
    #     api_key=OPENAI_API_KEY,
    #     model=EMBEDDING_MODEL,
    #     # 'dimensions' 파라미터를 사용하여 벡터 길이를 1536으로 고정합니다.
    #     dimensions=EMBEDDING_DIMENSION
    # )
    embedding_model = HuggingFaceEndpointEmbeddings(
        model=EMBEDDING_MODEL,
        # task="feature-extraction",
        huggingfacehub_api_token=HF_TOKEN,
    )

    print(f"✅ LangChain OpenAIEmbeddings ({EMBEDDING_MODEL}) 클라이언트가 초기화되었습니다.")

except Exception as e:
    print(f"❌ 클라이언트 초기화 실패: {e}")
    exit()

✅ Zilliz Cloud에 연결되었습니다.
✅ LangChain OpenAIEmbeddings (jhgan/ko-sroberta-nli) 클라이언트가 초기화되었습니다.


In [ ]:
# 필드 정의
pk_field = FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True)
text_field = FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=512)
# dim=1536은 text-embedding-3-small에 맞게 고정
vector_field = FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=EMBEDDING_DIMENSION)
description_field = FieldSchema(name="description", dtype=DataType.VARCHAR, max_length=65535)

# 컬렉션 스키마
schema = CollectionSchema(
    fields=[pk_field, text_field, vector_field, description_field],
    description="신한 관련 정보를 위한 text-embedding-3-small 컬렉션"
)

if milvus_client.has_collection(collection_name=COLLECTION_NAME):
    milvus_client.drop_collection(collection_name=COLLECTION_NAME)
    print(f"🔄 기존 컬렉션 '{COLLECTION_NAME}'을(를) 삭제했습니다.")



milvus_client.create_collection(
    collection_name=COLLECTION_NAME,
    schema=schema,
    metric_type="COSINE"
)


index_params = milvus_client.prepare_index_params()
index_params.add_index(
    field_name="vector",
    index_type="HNSW",  # 인덱스 유형: HNSW, IVFFLAT 등
    metric_type="COSINE" # 컬렉션 생성 시 지정한 메트릭과 일치해야 함
)
milvus_client.create_index(
    collection_name=COLLECTION_NAME,
    index_params=index_params,
    timeout=300 # 인덱스 생성에 시간이 걸릴 수 있으므로 타임아웃을 늘릴 수 있습니다.
)

In [ ]:
embeddings1 = embedding_model.embed_documents(store_nm_lst1)
embeddings2 = embedding_model.embed_documents(store_nm_lst2)

In [ ]:
data_to_insert1 = []
for text_a, vector, text_b in zip(store_nm_lst1, embeddings1, st_final_lst1):
    data_to_insert1.append({
        "text": text_a,
        "vector": vector,
        "description": text_b
    })

In [ ]:
data_to_insert2 = []
for text_a, vector, text_b in zip(store_nm_lst2, embeddings2, st_final_lst2):
    data_to_insert2.append({
        "text": text_a,
        "vector": vector,
        "description": text_b
    })

In [ ]:
print(len(data_to_insert1))
print(len(data_to_insert2))

2000
2185


In [ ]:
try:
    insert_result1 = milvus_client.insert(
        collection_name=COLLECTION_NAME,
        data=data_to_insert1
    )

    insert_result2 = milvus_client.insert(
        collection_name=COLLECTION_NAME,
        data=data_to_insert2
    )


except Exception as e:
    print(f"Error inserting data: {e}")

milvus_client.load_collection(collection_name=COLLECTION_NAME)
print(f"🔄 컬렉션 '{COLLECTION_NAME}' 로드 완료.")

🔄 컬렉션 'shinahn_collection_hf' 로드 완료.


여기서부터는 테스트를 위한 코드임

In [ ]:
search_query = "성우**"
query_vector = embedding_model.embed_query(search_query)
search_vectors = [query_vector]
milvus_client = MilvusClient(uri=ZILLIZ_URI, token=ZILLIZ_TOKEN)
output_fields = ["text", "description"]
search_result = milvus_client.search(
    collection_name=COLLECTION_NAME,
    data=search_vectors,      # 검색할 쿼리 벡터
    limit=1,                  # 상위 2개의 결과만 가져옴
    output_fields=output_fields # ⬅️ 검색 결과에 포함할 필드 지정
)

In [ ]:
search_result[0][0]['entity']['description']

'{\n    "가맹점구분번호": "16184E93D9",\n    "가맹점주소": "서울 성동구 마장동",\n    "가맹점명": "성우**",\n    "브랜드구분코드": NaN,\n    "가맹점지역": "서울 성동구",\n    "업종": "축산물",\n    "상권": "마장동",\n    "개설일": 20130320,\n    "폐업일": NaN,\n    "가맹점 운영개월수 구간": [\n        "10-25%",\n        "10-25%",\n        "10-25%",\n        "10-25%",\n        "10-25%",\n        "10-25%",\n        "10-25%",\n        "10-25%",\n        "10-25%",\n        "10-25%",\n        "10-25%",\n        "10-25%",\n        "10-25%",\n        "10-25%",\n        "10-25%",\n        "10-25%",\n        "10-25%",\n        "10-25%",\n        "10-25%",\n        "10-25%",\n        "10-25%",\n        "10-25%",\n        "10-25%",\n        "10-25%"\n    ],\n    "매출금액 구간": [\n        "25-50%",\n        "25-50%",\n        "25-50%",\n        "50-75%",\n        "25-50%",\n        "25-50%",\n        "25-50%",\n        "25-50%",\n        "10-25%",\n        "50-75%",\n        "50-75%",\n        "25-50%",\n        "25-50%",\n        "25-50%",\n        "25-50%",\n        

Retrieve 한 값을 google LLM 모델에 전송하여 결과 확인하기

In [ ]:
from google import genai
from google.genai.errors import APIError

gemini_client  = genai.Client()
MODEL_NAME = 'gemini-2.5-flash'

In [ ]:

def retrieve_and_format_rag_input_with_description(user_query: str) -> str:

  query_vector = embedding_model.embed_query(user_query)
  search_vectors = [query_vector]

  output_fields = ["text", "description"]
  search_result = milvus_client.search(
      collection_name=COLLECTION_NAME,
      data=search_vectors,      # 검색할 쿼리 벡터
      limit=1,                  # 상위 2개의 결과만 가져옴
      output_fields=output_fields # ⬅️ 검색 결과에 포함할 필드 지정
  )

  result = search_result[0][0]['entity']['description']
  return result
    # 1. 유사성 검색 수행
    # results = vectorstore.similarity_search(query=user_query, k=1)

    # return results[0].metadata['description']

In [ ]:
def generate_answer_with_description_rag(gemini_client, user_query: str):
    """
    검색된 컨텍스트(문서 내용 + description 메타데이터)를 기반으로 Gemini에게 답변을 요청합니다.
    """

    full_context = retrieve_and_format_rag_input_with_description(user_query )


    # 프롬프트 구성
    prompt = f"""
      아래의 데이터는 JSON 형식의 데이터로 특정 가맹점의 정보와 그 가맹점의 최근 24 개월간의 월별 이용 정보와 월별 이용 고객 정보 데이터이며 시간순으로 정렬이 되어 있어.
      - 값이 -999999.99 인 경우 정보가 존재하지 않는 경우임
      - '가맹점 운영 개월수 구간': 0% 에 가까울 수록 상위임(운영개월 수가 상위 임)
      - '매출금액 구간': 0% 에 가까울 수록 상위임(매출 금액이 상위 임)
      - '매출건수 구간': 0% 에 가까울 수록 상위임(매출 건수가 상위 임)
      - '유니크 고객 수 구간': 0% 에 가까울 수록 상위임(unique 고객 수가 상위임)
      - '객단가 구간': 0% 에 가까울 수록 상위임(객단가가 상위 임)
      - '취소율 구간': 1 구간에 가까울 수록 취소율이 낮음
      - '동일 업종 매출금액 비율': 동일 업종 매출 금액 평균 대비 해당 가맹점의 매출 금액 비율이며 평균과 동일할 경우 100 이야.
      - '동일 업종 매출건수 비율': 동일 업종 매출 건수 평균 대비 해당 가맹점의 매출 건수 비율이며 평균과 동일할 경우 100 이야.
      - '동일 업종 내 매출 순위 비율': ('업종 내 순위'/'업종 내 전체 가맹점'* 100) 을 계산한 값으로 0 에 가까울 수록 상위에 랭킹되는거야.
      - '동일 상권 내 매출 순위 비율': ('상권 내 순위'/'상권 내 전체 가맹점'* 100) 을 계산한 값으로 0 에 가까울 수록 상위에 랭킹되는거야.

      아래의 data 를 분석해서 해당 가맹점의 매출 전략을 제안해줘.

    {full_context}

    ---

    QUERY:
    {user_query}
    """

    response = gemini_client.models.generate_content(
        model=MODEL_NAME,
        contents=prompt,
        config={
            "temperature": 0.1 # 사실 기반 답변을 위해 낮은 온도를 사용
        }
    )

    return response.text


In [ ]:
user_query = "이혜*******"
final_answer = generate_answer_with_description_rag(
    gemini_client=gemini_client,
    user_query=user_query
)
print(f"--- 💡 Gemini의 최종 답변 (메타데이터 활용 RAG) ---\n{final_answer}")

--- 💡 Gemini의 최종 답변 (메타데이터 활용 RAG) ---
주어진 데이터를 바탕으로 '이혜*******' 가맹점의 현황을 분석하고, 이를 토대로 매출 전략을 제안합니다.

---

## '이혜*******' 가맹점 매출 전략 제안

### 1. 가맹점 현황 요약

*   **업종:** 인삼제품
*   **위치:** 서울 성동구 행당동 (행당 상권)
*   **개설일:** 2013년 12월 16일 (약 10년 이상 운영된 장수 가맹점)
*   **주요 문제점:**
    *   **매우 저조한 매출 성과:** 최근 24개월간 매출금액, 매출건수, 유니크 고객 수 모두 대부분 '90%초과(하위 10% 이하)' 또는 '75-90%' 구간에 머물러 있으며, 동일 업종 및 상권 내에서도 매우 낮은 순위를 기록하고 있습니다.
    *   **신규 고객 유치 부진:** 신규 고객 비중이 거의 0%에 가까워 새로운 고객 유입이 거의 없습니다.
    *   **제한적인 고객층:** 고객의 100%가 거주 인구이며, 직장인이나 유동인구 고객은 거의 없습니다. 주 고객층은 여성 40대(최근 60% 비중)와 여성 50대(20%), 남성 40대(15%)로 매우 편중되어 있습니다. 특히 남성 30대, 50대 고객 비중이 크게 감소했습니다.
    *   **미활용 채널:** 배달 매출이 전혀 없습니다.
*   **긍정적 요소:**
    *   **낮은 취소율:** 취소율이 '상위1구간'으로 매우 낮아, 일단 발생한 매출은 안정적입니다.
    *   **재방문 고객 비중:** 최근 33.33%로 비교적 높은 재방문율을 보입니다. 이는 기존 고객들의 만족도가 높다는 것을 시사합니다.
    *   **오랜 운영 기간:** 10년 이상 운영된 가맹점으로, 지역 내 인지도는 어느 정도 있을 수 있습니다.

### 2. 상세 분석 및 차트

#### 2.1. 매출 및 고객 성과 추이

아래 차트는 최근 24개월간 가맹점의 매출금액, 매출건수, 유니크 고객 수 구간을 보여줍니다

In [ ]:
[{'동대**': 'ㅇㄴㄹㅇㄴㄹㅇㅁㄴㄹㅇㄴㅁ' }, {'게키**':}]